**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from class_utils import show_tree

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Rozhodovacie stromy pre klasifikáciu

Ďalej ukážeme, ako sa dá klasifikátor na báze rozhodovacieho stromu aplikovať na dátovú množinu [Titanic](https://www.kaggle.com/c/titanic). Keďže sme s touto dátovou množinou už pracovali a vieme ako ju predspracovať, nebudeme to isté cvičenie opakovať znovu. Kód potrebný na načítanie a predspracovanie dát je v nasledujúcej bunke a kvôli stručnosti je skrytý.



In [ ]:
#@title -- Preprocessing the Data -- { display-mode: "form" }
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']
class_names = ["died", "survived"]

output = "Survived"

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values.reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values.reshape(-1)

### Učenie

Doslova jediná vec, ktorú treba v tejto chvíli zmeniť oproti predošlému príkladu je vymeniť triedu `KNeighborsClassifier` za `DecisionTreeClassifier`. Zvyšok kódu môže zostať rovnaký.



In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

### Testovanie

Kód na testovanie modelu sa dá tiež prebrať bezo zmeny.



In [ ]:
y_test = model.predict(X_test)

cm = pd.crosstab(Y_test, y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm, "\n")

acc = accuracy_score(Y_test, y_test)
print("Accuracy = {}".format(acc))

Správnosť, ktorú náš rozhodovací strom dosahuje, nie je príliš dobrá. Vlastne bude pravdepodobne horšia než to bolo v príklade s KNN. To je samozrejme veľmi podozrivé a môže to znamenať, že sa náš model preučil. Aby sme si overili, či je to naozaj tak, mali by sme model otestovať na tréningových dátach. Ak budú výsledky omnoho lepšie, znamená to, že sa model naozaj preučil a že jeho hyperparametre treba upraviť tak, aby sme znížili jeho kapacitu a získali model, ktorý bude zovšeobecňovať.



In [ ]:
y_train = model.predict(X_train)

acc_train = accuracy_score(Y_train, y_train)
print("Accuracy = {}".format(acc_train))

Ako sa ukazuje, správnosť na tréningových dátach je naozaj omnoho vyššia než na testovacích, čo indikuje preučenie. Môžeme si tiež vizualizovať výsledný stroj a preveriť, aký je zložitý. Použijeme na to pomocnú funkciu `show_tree`. Strom bude pravdepodobne pomerne zložitý a ťažko čitateľný.



In [ ]:
show_tree(model,
  feature_names=categorical_inputs+numeric_inputs,
  class_names=class_names)

### Ladenie hyperparametrov a lepšie prerezávanie

V ďalšom kroku si ukážeme, ako naladiť hyperparametre rozhodovacieho stromu tak, aby bol jednoduchší a aby sa predišlo preučeniu. Toto sa dá docieliť pomocou prerezávanie, ktoré sa delí na dva druhy:

* **dopredné prerezávanie**  (pre-pruning): ako strom rastie, zabráni sa vzniku niektorých vetvení za predpokladu, že nespĺňajú všetky stanovené kritériá;
* **spätné prerezávanie**  (post-pruning): nechá sa narásť úplný strom, z ktorého sa následne niektoré vetvenia odstraňujú.
V tomto príklade budeme používať dopredné prerezávanie a jeho parametre budeme špecifikovať v rámci konštruktoru triedy `DecisionTreeClassifier`.

#### Použitie krížovej validácie

Pri ladení hyperparametrov budeme potrebovať spôsob ako overiť, ktoré hodnoty dobre fungujú. Nemôžeme každú hodnotu jednoducho testovať na testovacích dátach: pripomeňme si, že testovacie dáta sa smú použiť len raz – na otestovanie finálneho modelu.

Máme v zásade 2 možnosti:

* Rozdeliť dátovú množinu na 3 časti: trénovaciu, validačnú a testovaciu časť (validačná časť sa použije na ladenie hyperparametrov a testovacia až na konci na overenie zovšeobecnenia finálneho modelu).
* Použiť krížovú validáciu: Tréningová množina sa rozdelí na $k$ častí. Mod l sa natrénuje na $k-1$ častiach a otestuje sa na zostávajúcej časti. 
To split the dataset into 3 parts: the training set, the validation set and the testing set (the validation set would be used to tune the hyperparameters and the testing set would be used at the end to verify that the final model generalizes). Tento proces sa zopakuje pre všetky kombinácie častí a výsledky sa spriemerujú.

Keďže učenie rozhodovacích stromov netrvá dlho a naša dátová množina nie je príliš veľká, v aktuálnom príklade použijeme krížovú validáciu. Pozrime sa ako by sa aplikovala pomocou balíčka `scikit-learn`. Použijeme funkciu `sklearn.model_selection.cross_validate` a určíme, že `cv=5`, čo znamená, že tréningovú množinu budeme deliť na $k=5$ častí. Funkcia navráti správnosť pre každú kombináciu a my z nich na konci vypočítame priemer, ktorý použijeme ako indikátor úspešnosti nášho modelu.



In [ ]:
cross_validate(model, X_train, Y_train, cv=5)['test_score'].mean()

#### Zmena hyperparametrov

Ďalej už poďme realizovať priamo ladenie hyperparametrov. Aby sme zistili, aké hyperparametre vieme vlastne pri konštrukcii rozhodovacieho stromu nastaviť, nahliadnime do dokumentácie.



In [ ]:
print(DecisionTreeClassifier.__doc__)

Minimálny počet vzoriek na list stromu (`min_samples_leaf`) vyzerá ako dobrý kandidát: ak predikciu robíme na základe veľmi malého počtu vzoriek, je pravdepodobné, že nebude reprezentatívna. Môžete samozrejme experimentovať aj s inými parametrami ako sú napr. maximálna hĺbka stromu a ďalšie.

---
#### Úloha 1: Ladenie `min_samples_leaf`

**V nasledujúcej bunke experimentujte s rôznymi hodnotami `min_samples_leaf` a pokúste sa maximalizovať správnosť z krížovej validácie. Sledujte tiež, aký vplyv má hodnota hyperparametra na štruktúru stromu.** 

---


In [ ]:
model = DecisionTreeClassifier(
    
    
    min_samples_leaf=    # ------
    
    
)

acc = cross_validate(model, X_train, Y_train, cv=5)['test_score'].mean()
print("Cross-validation accuracy = {}".format(acc))

# we need to fit the model before we plot it
model.fit(X_train, Y_train)
show_tree(model)

### Testovanie vyladeného stromu

Keď sme vyladili hyperparametre, môžeme konečne verifikovať, ako dobre náš finálny model vlastne zovšeobecňuje. Model s najlepšími hyperparametrami opätovne natrénujeme na celej tréningovej množine a overíme správnosť na testovacích dátach. Výsledky by mali teraz byť podstatne lepšie.



In [ ]:
model.fit(X_train, Y_train)

In [ ]:
y_test = model.predict(X_test)
accuracy_score(Y_test, y_test)